In [25]:
test_input = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

with open('day19.txt', 'rt') as f: raw_input = f.read()

In [118]:
from collections import namedtuple
from queue import deque

Part = namedtuple('Part', ['x', 'm', 'a', 's'])
Operation = namedtuple('Operation', ['key', 'opr', 'value', 'result'])

def parse_rules(raw_input: str) -> dict[str, list]:
    raw_rules = raw_input.strip().split('\n\n')[0].strip()
    rules = {}

    for row in raw_rules.splitlines():
        operations = []
        key = row.strip().split('{')[0]
        raw_operations = row.strip().split('{')[1].strip('}')
        for raw_operation in raw_operations.split(','):
            if ':' in raw_operation:
                result = raw_operation.split(':')[1]
                ropr = raw_operation.split(':')[0]
                if '<' in ropr:
                    opr = '__lt__'
                    okey = ropr.split('<')[0]
                    ovalue = int(ropr.split('<')[1])
                else:
                    opr = '__gt__'
                    okey = ropr.split('>')[0]
                    ovalue = int(ropr.split('>')[1])
                operations.append(Operation(key=okey, opr=opr, value=ovalue, result=result))
            else:
                operations.append(raw_operation)
        rules[key] = operations
    return rules

def parse_parts(raw_input: str) -> list[Part]:
    raw_parts = raw_input.strip().split('\n\n')[1]
    
    parts = []
    for row in raw_parts.strip().splitlines():
        row = row[1:-1]
        kwargs = {}
        for ropr in row.split(','):
            key, value = ropr.split('=')
            kwargs[key] = int(value)
        parts.append(Part(**kwargs))
    return parts

def check_accepted(rules, part: Part):
    rule_stacks = deque()
    rule_stacks.append(('in', rules['in']))
    closed_stacks = set()

    while rule_stacks:
        rk, rule = rule_stacks.popleft()
        if rk in closed_stacks:
            print('Duplicated')
        result = None
        for operation in rule[:-1]:
            if isinstance(operation, Operation):
                operation_result = getattr(getattr(part, operation.key), operation.opr)(operation.value)
                if operation_result:
                    result = operation.result
                    break
        if not result:
            result = rule[-1]
        if result in ['A', 'R']:
            return result == 'A'
        
        closed_stacks.add(rk)
        rule_stacks.append((result, rules[result]))
        
    return False

def total_part(part: Part):
    return part.x + part.m + part.a + part.s

def solve1(raw_input: str):
    rules = parse_rules(raw_input)
    parts = parse_parts(raw_input)
    
    accepted_parts = [part for part in parts if check_accepted(rules, part)]
    total = sum([total_part(part) for part in accepted_parts])
    return total

In [119]:
solve1(raw_input)

401674

In [234]:
def reverse_operation(opr: Operation):
    if opr.opr == '__gt__':
        ropr = '__lt__'
        value = opr.value + 1
    else:
        ropr = '__gt__'
        value = opr.value - 1
    return Operation(key=opr.key, opr=ropr, value=value, result=None)

def flat_rules(rules):
    flat_rules = []
    stacks = deque()
    stacks.append(('in', []))
    
    while stacks:
        key, pre_oprs = stacks.popleft()
        if key == 'A':
            flat_rules.append(pre_oprs)
            continue
        elif key == 'R':
            continue
        oprs = rules[key]
        for opr_index, opr in enumerate(oprs[:-1]):
            current_oprs = [reverse_operation(i) for i in oprs[:opr_index]]
            current_oprs.append(opr)
            stacks.append((opr.result, [*pre_oprs, *current_oprs]))
        
        current_oprs = [reverse_operation(i) for i in oprs[:-1]]
        stacks.append((oprs[-1], [*pre_oprs, *current_oprs]))
    
    return flat_rules

def count_availables(flat_rules):
    case = 0
    for tfrule in flat_rules:
        total = 1
        for factor in 'xmas':
            oprs = [opr for opr in tfrule if opr.key == factor]
            if oprs:
                set_oprs = [
                    set(range(1, min(opr.value, 4001))) 
                    if opr.opr == '__lt__' else 
                    set(range(max(opr.value + 1, 1), 4001))
                    for opr in oprs
                ]
                availables = set(range(1, 4001))
                for soprs in set_oprs:
                    availables &= soprs
                total_availables = len(availables)
            else:
                total_availables = 4000
            total *= total_availables
        case += total
    return case

def solve2(raw_input):
    rules = parse_rules(raw_input)
    frules = flat_rules(rules)
    return count_availables(frules)

In [235]:
solve2(raw_input)

134906204068564